In [1]:
from datatree import DataTree
import dask.bag as db
import xarray as xr
import pandas as pd
import coiled
import pandas as pd
from fsspec.implementations.reference import LazyReferenceMapper
import os
import dask
import fsspec
import ujson
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

# Try out autoscaling? Set workers to # of refs?
# cluster = coiled.Cluster(n_workers=10)
# client = cluster.get_client()

In [2]:
cluster = coiled.Cluster(n_workers=10)
client = cluster.get_client()

Output()

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                              ╷                                                                   │
│   Package                    │ Note                                                              │
│ ╶────────────────────────────┼─────────────────────────────────────────────────────────────────╴ │
│   coiled_local_site_packages │ Source wheel built from                                           │
│                              │ /Users/nrhagen/.local/lib/python3.10/site-packages                │
│                              ╵                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [3]:

def _nasa_nex_df() -> pd.DataFrame:
    return pd.read_csv(
        "s3://carbonplan-share/nasa-nex-reference/nasa_nex_formatted.csv"
    )
    # specify ensemble_member doesn't change # in query (ie, choosing a gcm + scenario gives a unique ensemble member)

def _GCM_scenarios(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.read_csv(
        "s3://carbonplan-share/nasa-nex-reference/nasa_nex_formatted.csv")
    colapsed_df = (
        df.groupby(["GCM", "scenario", "ensemble_member"])["variable"]
        .apply(list)
        .reset_index()
    )
    colapsed_df["variable"] = colapsed_df["variable"].apply(lambda x: list(set(x)))

    return colapsed_df

def read_catalog_file(catalog_url):
    return pd.read_csv(catalog_url)


In [4]:
catalog_file_url = "s3://carbonplan-share/nasa-nex-reference/reference_catalog_prod.csv"
nasa_nex_df = _nasa_nex_df()
nasa_nex_catalog = _GCM_scenarios(nasa_nex_df) 
nasa_nex_catalog['ID'] = nasa_nex_catalog['GCM'] + '_' + nasa_nex_catalog['scenario']
kerchunk_ref_catalog = read_catalog_file(catalog_file_url)


## TESTING REMOVE FOR RUNNING!!!!
## TESTING REMOVE FOR RUNNING!!!!
kerchunk_ref_catalog = kerchunk_ref_catalog.sample(154)
## TESTING REMOVE FOR RUNNING!!!!
## TESTING REMOVE FOR RUNNING!!!!


missing_refs_df = nasa_nex_catalog.merge(kerchunk_ref_catalog, how='outer', on='ID',indicator=True).query('_merge != "both"')[['GCM', 'scenario', 'ID']]


In [5]:
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")

def build_reference_catalog(catalog_file_url: str):
    ref_list = ["s3://" + ref for ref in fs_read.ls('s3://carbonplan-share/nasa-nex-reference/references_prod/')]
    ref_list.remove("s3://carbonplan-share/nasa-nex-reference/references_prod/")
    ref_df = pd.DataFrame({"ID":None, "url":ref_list})
    ref_df['ID'] = ref_df['url'].str.split('/',expand=True)[5]
    ref_df.to_csv(catalog_file_url, index=False)

In [6]:

def gen_ref(row: pd.Series):

    GCM = row["GCM"]
    scenario = row["scenario"]

    file_pattern = nasa_nex_df.query(f"GCM == '{GCM}'  & scenario == '{scenario}'"
        )
    # generate refs for all files
    
    def generate_json_reference(fil):
        with fs_read.open(fil, **so) as infile:
            h5chunks = SingleHdf5ToZarr(infile, fil, inline_threshold=300)
            return h5chunks.translate()  # outf
    
    # def write_multi_ref(refs: list):

    #     fs = fsspec.filesystem("s3")
    #     target_root = "s3://carbonplan-share/nasa-nex-reference/references_prod/"
    #     store_name = f"{GCM}_{scenario}"
    #     output_file_name = "reference.parquet"
    #     outpath = target_root + store_name + "/" + output_file_name

    #     if fs.exists(outpath):
    #         fs.rm(outpath, recursive=True)
    #     fs.makedir(outpath)

    #     out = LazyReferenceMapper.create(1000, outpath, fs)

    #     MultiZarrToZarr(
    #         refs,
    #         remote_protocol="s3",
    #         concat_dims=["time"],
    #         identical_dims=["lat", "lon"],
    #         out=out,
    #     ).translate()

    #     out.flush()
    # # MultiZarrToZarr

    # # Write Parquet
    refs = [generate_json_reference(fil) for fil in file_pattern['url'].to_list()]
    # write_multi_ref(refs)





In [7]:
row_list = [row for _, row in missing_refs_df.iterrows()]


bag = db.from_sequence(row_list, npartitions=len(row_list)).map(gen_ref)

In [8]:
bag.compute()

/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 15.36 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


[None, None, None, None, None, None, None, None, None, None]

2023-11-15 19:16:46,330 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2023-11-15 19:54:09,415 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
TimeoutError: [Errno 60] Operation timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/comm/core.py", line 342, in connect
    comm = await wait_for(
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/utils.py", line 1910, in wait_for
    return await asyncio.wait_for(fut, timeout)
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/asyncio/tasks.py", line 445, in wait_for
    return fut.result()
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/comm/tcp.py", line 503, in conne

In [12]:
client.shutdown()

In [ ]:

ds_list = [ds.to_dataset() for ds in dt.leaves if ds.dims]
bag = db.from_sequence(ds_list, npartitions=len(ds_list)).map(calc_wbgt)

## Update Catalog

In [ ]:
# build_reference_catalog(catalog_file_url)